In [1]:
#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)

In [5]:
SOS_TOKEN =0
EOS_TOKEN=1

class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word={0:'SOS',1:'EOS'}
        self.n_words =2
    
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.n_words #Index of the said word
            self.index2word[self.n_words]=word
            self.n_words+=1
            self.word2count[word]=1
        else:
            self.word2count[word]+=1             

In [6]:
#Turn unicode to ASCII
def unicode2Ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c) != 'Mn'
    )
#lowercase, trim and remove punctuation

def normalizeString(s):
    s = unicode2Ascii(s.lower().strip())
    s = re.sub(r"([.!?])",r"\1",s)
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    return s 

In [9]:
#Read data file by splitting file into lines and then lines into pairs

def readLangs(lang1,lang2,reverse=False):
    path = '/home/cds/Desktop/rESUME PROJECTS/Machine Translation/data/eng-fra.txt'
    lines = open(path,encoding='utf-8').read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')]for l in lines]

    if reverse:
        pairs=[list(reversed(p))for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

In [12]:
#Filtering sentences by only taking sentences with max length less than 10
MAX_LENGTH =10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return[pair for pair in pairs if filterPair(pair)]

In [10]:
def prepareData(lang1,lang2,reverse=False):
    input_lang,output_lang,pairs = readLangs(lang1,lang2,reverse)
    print("Total sentence pairs ", len(pairs))
    pairs = filterPairs(pairs)
    print("Pairs retained", len(pairs))
    print("Counting words")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name,output_lang.n_words)
    return input_lang, output_lang, pairs

In [15]:
input_lang, output_lang,pairs = prepareData('eng','fra',True)
print(random.choice(pairs))

Total sentence pairs  135842
Pairs retained 11449
Counting words
fra 5713
eng 3802
['vous desobeissez aux ordres.', 'you re disobeying orders.']


In [16]:
#Seq2Seq model
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)

    def forward(self,input,hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded
        output,hidden=self.gru(output,hidden)
        return output,hidden
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device=device)
    

In [18]:
#The decoder
class DecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self,input,hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        output = self.softmax(self.out(output[0]))
        return output,hidden 
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size,device=device)

In [19]:
#Decoder with attention
class AttnDecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size,dropout_p=0.1,max_length=MAX_LENGTH):
        super(AttnDecoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size,hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine= nn.Linear(self.hidden_size * 2, self.max_length)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.linear(self.hidden_size,self.output_size)
    
    def forward(self,input,hidden,encoder_outputs):
        embedded = self.embedding(input).view(1,1,-1)
        embedded =self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0],hidden[0]),1)),dim=1
        )

        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        output = F.log_softmax(self.out(output[0]),dim=1)
        return output,hidden,attn_weights
    
    def inintHidden(self):
        return torch.zeros(1,1,self.hidden_size,device=device)


In [20]:
#prepare training data
def indexesFromSentence(lang,sentence):
    return[lang.word2index[word]for word in sentence.split(' ')]

def tensorFromSentence(lang,sentence):
    indexes = indexesFromSentence(lang,sentence)
    indexes.append(EOS_TOKEN)
    return torch.tensor(indexes,dtype=torch.long, device=device).view(-1,1)

def tensorsFrompair(pair):
    input_tensor = tensorFromSentence(input_lang,pair[0])
    target_tensor = tensorFromSentence(output_lang,pair[1])
    return(input_tensor, target_tensor)


In [ ]:
#Training
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder,decoder,encoder_optimizer,decoder_optimizer,criterion,max_length=MAX_LENGTH):
    